# Communicate with your digital twin: Edge harvester
This is a enterprise application which allows to communicate with the edge harvester launched in the notebook 08e via S³I-B protocol. This Notebook acts as an application in terms of the S³I and a client in terms of OAuth2 authentication.
Just go to the **Cell** drop-down menu and click on the **Run All** button.

In [ ]:
import s3i
import json
import getpass
import uuid
import os 
from tools import print_with_timestamp, yes, no
import ml 

In [ ]:
%%html
<img src="harvester_images/folie_2_3.png", width=1000, height=1000>

## Configure the notebook
In order to assign this notebook to your private application it needs a client id and the respective secret. In the following input fields, you can enter the id and the secret of your application.

In [ ]:
print_with_timestamp("Assign a client to this notebook. (The id of your application)")
app_id = input('[S3I]: Please enter your application id:').strip('," ')
app_secret = getpass.getpass('[S3I]: Please enter your application secret:').strip('," ')
print_with_timestamp("application id and secret are set.")

Next, you have to enter your username and password to authenticate yourself in S³I. 

In [ ]:
print_with_timestamp("This is a enterprise application, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your username and password are set.")

## Build and load
Analogously to the harvester, a application can also be created and launched using fml40 reference implementation library. All defined roles, functionalities and values etc. are respectively implemented according to the fml40 language. Next, a JSON file for the application is created and loaded into the notebook.

In [ ]:
app_name = "my_app_edge"
config_path = os.path.abspath(os.path.join("", "configs"))
config_file_name = ml.make_thing_config(thing_id=app_id, name=app_name, roles=[{"class": "ml40::App"}],
                                     config_path=config_path)
app_model = ml.load_config(config_filepath=os.path.join(config_path, config_file_name))
app = ml.create_thing(
                    model=app_model, grant_type="password", secret=app_secret,
                    username=username, password=password, 
                    is_broker=True, is_repo=False, is_broker_rest=True
                   )

## Setup a loger for the App

In [ ]:
ml.setup_logger(app_name)

## Run the App 
The created App is going to be launched in the following step. When the run_forever function is called, the App is started, and meanwhile its listener is also started to receive the messages sent to it via S³I Broker. Additionally, you can specific the receiver, with whom it will communicate. 

In [ ]:
app.run_forever()
config_path = os.path.abspath(os.path.join("", "configs"))
harvester_name = input("[S³I]: Please enter the name of your harvester: (my_edge_harvester) ")
cred_filepath = os.path.join(config_path, "{}_cred.json".format(harvester_name))
with open(cred_filepath) as file:
    cred = json.load(file)
receiver = cred.get("identifier")
app_endpoint = ml.find_broker_endpoint(app.dir, app_id)
receiver_endpoint = ml.find_broker_endpoint(app.dir, thing_id=receiver)

## Get the directory entry of the edge harvester

In [ ]:
%%html
<img src="harvester_images/dir.png", width=300, height=300>

In [ ]:
print_with_timestamp("The directory entry of the edge harvester: " + json.dumps(app.dir.queryThingIDBased(thingID=receiver), indent=2))

## Communicate with your edge harvester
The following steps refer to the communication the the created edge harvester using S³I-B protocol.  

In [ ]:
%%html
<img src="harvester_images/edge.png", width=500, height=500>

## Send a S³I-B ServiceRequest to your edge harvester

In the following, we will show you how to edit a S3I-B ServiceRequest to call a service function in your edge harvester. As prepared you can ask your harvester to accept the felling job, query the current status of your felling job and remove a felling job from the job list.

### Edit a felling job
A felling job can be created using python reference implementation as well. We will send the job to the edge harvester within a service request.

In [ ]:
subFeatures = [{
            "class": "fml40::Assortment",
            "grade": "fl",
            "name": "Stammholz Abschnitte",
            "subFeatures": [
                {
                    "class": "fml40::ThicknessClass",
                    "name": ">"
                },
                {
                    "class": "fml40::WoodQuality",
                    "name": "B-C"
                },
                {
                    "class": "fml40::HarvestingParameters",
                    "cuttingLengths": 20
                },
                {
                    "class": "fml40::TreeType",
                    "name": "Spruce",
                    "conifer": True
                },
                {
                    "class": "fml40::HarvestedVolume",
                    "volume": 140
                }
            ]
        }]
feature_config_json = ml.make_feature_config(class_name="fml40::FellingJob", subFeatures=subFeatures)
felling_job = ml.build_feature(feature=feature_config_json)

print_with_timestamp("Felling job created:\n" + json.dumps(felling_job.to_json(), indent=2))

### Make a S³I-B SerivceRequest

In [ ]:
def make_ser_param_type():
    """
    This function is used to make the service parameters and type for a S³I-B service request
    """
    while True:
        service_type = input('[S3I]: What kind of service request would you like to choose? \n<fml40::AcceptsFellingJobs/acceptJob> \n<fml40::AcceptsFellingJobs/queryJobStatus> \n<fml40::AcceptsFellingJobs/removeJob> \n')
        if "fml40::AcceptsFellingJobs/acceptJob" in service_type: 
            parameters = {"job": felling_job.to_json()}
        elif "fml40::AcceptsFellingJobs/queryJobStatus" in service_type:
            job_id = input("[S³I]: Please enter the job id to query the job status: ")
            parameters={"identifier": job_id}
        elif "fml40::AcceptsFellingJobs/removeJob" in service_type:
            job_id = input("[S³I]: Please enter the job id to delete the job: ")
            parameters={"identifier": job_id}
        else:
            print_with_timestamp("[S³I]: Error in service type, please rewrite it!")
            parameters = None
        
        if parameters is None:
            continue
        else:
            return parameters, service_type
        
def prepare_service_req():
    serv_req = s3i.messages.ServiceRequest()
    parameters, service_type = make_ser_param_type()
    serv_req.fillServiceRequest(
        senderUUID=app_id, receiverUUID=[receiver], sender_endpoint=app_endpoint,
        serviceType=service_type,
        parameters=parameters,
        msgUUID="s3i:{}".format(uuid.uuid4())
    )
    return serv_req.msg


### Send the S³I-B ServiceRequest

In [ ]:
resp = app.broker.send([receiver_endpoint], json.dumps(prepare_service_req()))
while True: 
    send_serv_again = input("[S3I]: Do you want to send a service request again(j/n): ")
    if send_serv_again in yes:
        resp = app.broker.send([receiver_endpoint], json.dumps(prepare_service_req()))
    elif send_serv_again in no:
        break

## Send a S³I-B GetValueRequest to your edge harvester
In addition, the edge harvester is supposed to handle a get value request. As below we prepare and send a S³I-B GetValueRequest to the harvester and wait for the corresponding GetValueReply. 

In [ ]:
def prepare_get_value_req():
    getv_req = s3i.GetValueRequest()
    path = input("[S³I]: Please enter the attribute path that you want to query: \ne.g. [] \n[attributes/name] \n[attributes/features/ml40::Location/longitude] \n[attributes/features/ml40::Composite/targets/ml40::Engine/features/ml40::RotationalSpeed/rpm] ")
    getv_req.fillGetValueRequest(
        senderUUID=app_id, receiverUUID=[receiver], sender_endpoint=app_endpoint,
        attributePath=path, msgUUID="s3i:{}".format(uuid.uuid4())
    )
    receiver_endpoint = ml.find_broker_endpoint(app.dir, thing_id=receiver)
    return getv_req.msg

app.broker.send([receiver_endpoint], json.dumps(prepare_get_value_req()))
while True: 
    send_getv_again = input("[S3I]: Do you want to send a get value request again(j/n): ")
    if send_getv_again in yes:
        app.broker.send([receiver_endpoint], json.dumps(prepare_get_value_req()))
    elif send_getv_again in no:
        break